In [1]:
# import os
# import sys

# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

In [2]:
from numba.experimental import jitclass
from numba import njit
from numba.core import types
from numba import float64
import pandas as pd
import numpy as np

In [10]:
spec = {
    "helper_class_self_num": types.float_,
    "helper_class_self_var": types.float_,
}


class EmptyClass:
    def __init__(
        self,
        helper_class_self_num,
        helper_class_self_var,
    ):
        self.helper_class_self_num = helper_class_self_num
        self.helper_class_self_var = helper_class_self_var

    def calc_num_one(self, helper_num):
        pass


@jitclass(spec)
class SubClassOne(EmptyClass):
    def __init__(
        self,
        helper_class_self_num,
        helper_class_self_var,
    ):
        self.helper_class_self_num = helper_class_self_num
        self.helper_class_self_var = helper_class_self_var

    def calc_num_one(self, helper_num):
        return helper_num / self.helper_class_self_num


class SubTwo(EmptyClass):
    def __init__(self, helper_class_self_num, helper_class_self_var):
        super().__init__(helper_class_self_num, helper_class_self_var)

    def calc_num_one(self, helper_num):
        return pd.DataFrame([helper_num])

In [11]:
@njit(cache=True)
def tester_func(calculator: EmptyClass):
    return calculator.calc_num_one(40)

calculator=SubClassOne(10, 20)
tester_func(calculator=calculator)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1mUntyped global name 'super':[0m [1m[1mCannot determine Numba type of <class 'type'>[0m
[1m
File "C:\Users\User\AppData\Local\Temp\ipykernel_31636\1528219533.py", line 23:[0m
[1m    def __init__(self, helper_class_self_num, helper_class_self_var):
[1m        super().__init__(helper_class_self_num, helper_class_self_var)
[0m        [1m^[0m[0m
[0m
[0m[1mDuring: resolving callee type: jitclass.SubClassOne#2dde595fdf0<helper_class_self_num:float32,helper_class_self_var:float32>[0m
[0m[1mDuring: typing of call at <string> (3)
[0m
[0m[1mDuring: resolving callee type: jitclass.SubClassOne#2dde595fdf0<helper_class_self_num:float32,helper_class_self_var:float32>[0m
[0m[1mDuring: typing of call at <string> (3)
[0m
[1m
File "<string>", line 3:[0m
[1m<source missing, REPL/exec in use?>[0m


In [1]:
import numpy as np
from numba import int64, types, njit
from numba.experimental import jitclass


spec = {'data': types.float_}


@jitclass(spec)
class Foo():
    def __init__(self, init_data):
        self.data = init_data

    def calc(self, num):
        pass


@jitclass()
class Bar:
    foo_inst: Foo

    def __init__(self, foo_data):
        # This constructs the Foo instance in the Bar constructor
        self.foo_inst = Foo(foo_data)

    def calc(self, num):
        return num * self.foo_inst.data


@jitclass()
class Baz:
    foo_inst: Foo

    def __init__(self, foo_inst):
        # This assigns a reference to the Foo instance in the Bar constructor
        self.foo_inst = foo_inst


@njit(cache=True)
def testing_yes():
    x = 30.0
    # Build a Bar instance
    bar = Bar(x)
    print(bar.foo_inst.data)
    print(bar.calc(10))


testing_yes()

30.0
300.0
